In [1]:
import os
import re
import json
import pprint
import datetime
import pandas as pd

#pp = pprint.PrettyPrinter(indent=4)


In [5]:
data = []
with open('/home/hm/Desktop/bluecap/bcpnews/newspaper_scraper/articles_elconfidencial.json') as input_file:
    for line in input_file:
        data.append(json.loads(line))

In [6]:
df = pd.DataFrame(data)
df.head(5)

,authors,keywords,publish_date,summary,text,title,top_image,url
0,"[Marcos Lamelas. Barcelona, Antonio Fernández....","[vacaciones, se, que, sus, estudia, para, ulti...",2017-05-28 00:00:00,La Generalitat está estudiando que el mes de a...,La Generalitat está estudiando que el mes de a...,Vacaciones: La Generalitat estudia dejar a sus...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/cataluna...
1,"[Carlos Sánchez, Contacta Al Autor]","[se, que, cataluña, es, una, abusarás, pacienc...",2017-05-28 00:00:00,La conclusión a la que llegó fue que lo determ...,En la célebre conferencia que dictó Ernest Ren...,"Geografía: ¿Hasta cuándo, Cataluña, abusarás d...",https://www.ecestaticos.com/imagestatic/clippi...,https://blogs.elconfidencial.com/espana/mientr...
2,"[Antonio Fernández. Sitges, José Antonio Zarza...","[sea, se, que, rajoy, es, una, mientras, el, v...",2017-05-27 00:00:00,Lo hizo en el transcurso de su discurso de cla...,"El presidente del Gobierno español, Mariano Ra...","Independencia de Cataluña: Rajoy, en Barcelona...",https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...
3,"[Juanma Romero, Isabel Morillo. Sevilla, Conta...","[se, que, al, primarias, error, sánchez, el, l...",2017-05-28 00:00:00,Pero Sánchez va mucho más allá: sostiene que e...,"La victoria no es todavía completa, aunque muy...",Primarias PSOE: El PSOE de Sánchez: Estado plu...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...
4,"[Juanma Romero, Víctor Romero. Valencia, Isabe...","[se, sigue, que, primarias, marca, sánchez, pa...",2017-05-27 00:00:00,Pero entre los 69 compromisarios elegidos en C...,Ximo Puig no será el único presidente autonómi...,Primarias PSOE: Page sigue a Puig y marca dist...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...


In [4]:
"number of rows in jason file is " + str( df.shape[0] )

'number of rows in jason file is 3213'

In [7]:
urls = df.url
urls.head(5)

0    https://www.elconfidencial.com/espana/cataluna...
1    https://blogs.elconfidencial.com/espana/mientr...
2    https://www.elconfidencial.com/espana/2017-05-...
3    https://www.elconfidencial.com/espana/2017-05-...
4    https://www.elconfidencial.com/espana/2017-05-...
Name: url, dtype: object

In [8]:
#  how manay unique article body text 
text = list(set(df.text))
len(text)

3121

In [9]:
print("The number of rows in scrapy jason output file is", df.shape[0], ".")
print("However, only", len(text), "unique urls are in them.")

The number of rows in scrapy jason output file is 3214 .
However, only 3121 unique urls are in them.


In [10]:
df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"}).head(10)

,text,count
0,"BIOGRAFÍA\n\nTrader, empezó su carrera profesi...",20
1,,4
2,"Entrevista a Attila Szabo\n\n""Un 3% de los run...",4
3,"5 de 15\n\nEn la imagen, Villa Sola, en Tremez...",2
4,Resido en el centro de Madrid y realizó con fr...,2
5,Me retiraron el carnet 8 meses por una condena...,2
6,Mi casera me sube el precio del alquiler 150 e...,2
7,"Mi coche tiene tracción a las cuatro ruedas, e...",2
8,Dos homosexuales indonesios recibieron este ma...,2
9,Mi hijo está viviendo en Argentina desde hace ...,2


In [11]:
# Lets look at the text and url of the most frequent article text
idx = df.index[ df.text.str.contains('BIOGRAFÍA\n\nTrader, empezó su carrera')]
print(df.loc[idx[0]].url)
print(df.loc[idx[0]].text)

https://blogs.elconfidencial.com/mercados/bolsagora/2017-04-13/asi-cierra-mercado-bancos-mostrar-deterioro_1366748/
BIOGRAFÍA

Trader, empezó su carrera profesional en GVC en 1995. Es fundador de Ágora Asesores Financieros EAFI y uno de los analistas técnicos más reputados del panorama español. Ha colaborado con diversos medios de comunicación, entre los que se encuentra El Economista, desde su fundación hasta julio de 2014. Desde septiembre de 2014 se encarga de dirigir a la comunidad inversora de El Confidencial.


In [12]:
# remove all of the articles like above
df = df.drop(idx)

In [13]:
# Lets look at the text and url of the articles that repeated 4 times
idx = df.index[ df.text.str.contains('Entrevista a Attila Szabo\n\n"Un 3% de los')]
print(df.loc[idx[0]].url)
print(df.loc[idx[0]].text)

https://www.elconfidencial.com/alma-corazon-vida/running/2016-09-17/running-proyecto-documental-portada_1260314/
Entrevista a Attila Szabo

"Un 3% de los runners son adictos, del mismo modo que lo es un adicto al tabaco o a la heroína"


In [14]:
# again lets remove all of the articles like above because they are not really an article
df = df.drop(idx)

In [15]:
# remove all the articles without any text
df = df.drop( df[df['text']==''].index)

In [16]:
#removing the duplicate articles based on their text
df.drop_duplicates('text','first',True)
df.shape[0]

3118

In [17]:
# extracting source column from the url of the articles
l = df['url'].apply(lambda x: re.findall('(?:\/\/\w+\.\w+\.\w+\/)(\w+)\/\w',x))
flat_list = []

for sublist in l:
    for item in sublist:
        flat_list.append(item)
df['source'] = pd.DataFrame(flat_list) 
df.head(5)
#print("\n".join(flat_list[0:10])) 

,authors,keywords,publish_date,summary,text,title,top_image,url,source
0,"[Marcos Lamelas. Barcelona, Antonio Fernández....","[el, por, se, que, como, generalitat, ultimar,...",2017-05-28 00:00:00,La Generalitat está estudiando que el mes de a...,La Generalitat está estudiando que el mes de a...,Vacaciones: La Generalitat estudia dejar a sus...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/cataluna...,espana
1,"[Carlos Sánchez, Contacta Al Autor]","[nuestra, el, abusarás, que, la, se, hasta, lo...",2017-05-28 00:00:00,La conclusión a la que llegó fue que lo determ...,En la célebre conferencia que dictó Ernest Ren...,"Geografía: ¿Hasta cuándo, Cataluña, abusarás d...",https://www.ecestaticos.com/imagestatic/clippi...,https://blogs.elconfidencial.com/espana/mientr...,espana
2,"[Antonio Fernández. Sitges, José Antonio Zarza...","[el, rajoy, presidente, que, la, se, sea, cele...",2017-05-27 00:00:00,Lo hizo en el transcurso de su discurso de cla...,"El presidente del Gobierno español, Mariano Ra...","Independencia de Cataluña: Rajoy, en Barcelona...",https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana
3,"[Juanma Romero, Isabel Morillo. Sevilla, Conta...","[el, se, su, plurinacional, estado, la, sánche...",2017-05-28 00:00:00,Pero Sánchez va mucho más allá: sostiene que e...,"La victoria no es todavía completa, aunque muy...",Primarias PSOE: El PSOE de Sánchez: Estado plu...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana
4,"[Juanma Romero, Víctor Romero. Valencia, Isabe...","[el, por, su, se, la, que, sánchez, distancias...",2017-05-27 00:00:00,Pero entre los 69 compromisarios elegidos en C...,Ximo Puig no será el único presidente autonómi...,Primarias PSOE: Page sigue a Puig y marca dist...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana


In [18]:
# extracting the section/source of the articles (useful for filtering unrelated sources)
df_source = df.groupby('source').size().sort_values(0, ascending=False).reset_index().rename(columns={0:"count"})
df_source.head(100)

,source,count
0,espana,623
1,deportes,315
2,multimedia,300
3,empresas,183
4,cultura,173
5,tecnologia,164
6,mundo,137
7,mercados,107
8,economia,90
9,motor,67


In [21]:
# 
df.groupby('publish_date').size().sort_values(0, ascending=False).reset_index().rename(columns={0:'count'}).head(10)


,publish_date,count
0,2018-04-05 00:00:00,69
1,2017-04-18 00:00:00,68
2,2017-04-26 00:00:00,67
3,2017-06-13 00:00:00,63
4,2017-09-25 00:00:00,63
5,2018-03-20 00:00:00,63
6,2017-10-19 00:00:00,62
7,2018-03-12 00:00:00,61
8,2018-01-23 00:00:00,60
9,2017-11-28 00:00:00,59


In [22]:
#split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str#, columns = ['x' ,'y'])

df.drop(['time', 'publish_date'], axis = 1, inplace= True)

In [23]:
#convert date columns to datetime and create day_of_week column

df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name
df.head()

,authors,keywords,summary,text,title,top_image,url,source,date,day_of_week
0,"[Marcos Lamelas. Barcelona, Antonio Fernández....","[el, por, se, que, como, generalitat, ultimar,...",La Generalitat está estudiando que el mes de a...,La Generalitat está estudiando que el mes de a...,Vacaciones: La Generalitat estudia dejar a sus...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/cataluna...,espana,2017-05-28,Sunday
1,"[Carlos Sánchez, Contacta Al Autor]","[nuestra, el, abusarás, que, la, se, hasta, lo...",La conclusión a la que llegó fue que lo determ...,En la célebre conferencia que dictó Ernest Ren...,"Geografía: ¿Hasta cuándo, Cataluña, abusarás d...",https://www.ecestaticos.com/imagestatic/clippi...,https://blogs.elconfidencial.com/espana/mientr...,espana,2017-05-28,Sunday
2,"[Antonio Fernández. Sitges, José Antonio Zarza...","[el, rajoy, presidente, que, la, se, sea, cele...",Lo hizo en el transcurso de su discurso de cla...,"El presidente del Gobierno español, Mariano Ra...","Independencia de Cataluña: Rajoy, en Barcelona...",https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana,2017-05-27,Saturday
3,"[Juanma Romero, Isabel Morillo. Sevilla, Conta...","[el, se, su, plurinacional, estado, la, sánche...",Pero Sánchez va mucho más allá: sostiene que e...,"La victoria no es todavía completa, aunque muy...",Primarias PSOE: El PSOE de Sánchez: Estado plu...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana,2017-05-28,Sunday
4,"[Juanma Romero, Víctor Romero. Valencia, Isabe...","[el, por, su, se, la, que, sánchez, distancias...",Pero entre los 69 compromisarios elegidos en C...,Ximo Puig no será el único presidente autonómi...,Primarias PSOE: Page sigue a Puig y marca dist...,https://www.ecestaticos.com/imagestatic/clippi...,https://www.elconfidencial.com/espana/2017-05-...,espana,2017-05-27,Saturday


In [24]:
#number of articles in different days of the week
df.groupby('day_of_week').size().sort_values(0, ascending=False).reset_index().rename(columns={0:'count'}).head(10)

,day_of_week,count
0,Tuesday,553
1,Wednesday,523
2,Thursday,483
3,Monday,448
4,Saturday,397
5,Friday,382
6,Sunday,330
